# Implementation of the Pressure model

In [ ]:
#Main Packages
import pandas as pd
import numpy as np
import json
import seaborn as sns
import itertools

#Plotting
import matplotlib.pyplot as plt
from xgboost import plot_importance

#Statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf

#Sklearn metrics
from sklearn.metrics import roc_curve, f1_score, roc_auc_score, precision_recall_curve, auc, accuracy_score,confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#Imbalanced learning package for synthetic data
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids

#Model Packages
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

#### Choose dataset to load

In [ ]:
data = pd.read_csv("FixedRBF.csv")

In [ ]:
data

#### Correlation matrix

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
x = data.drop(['SuccessFlag','Frame'], axis=1)
y = data['SuccessFlag']

In [ ]:
sns.scatterplot(x= 'DistanceGoal', 
           y= 'MinDistance', 
           data=data, 
           hue= 'SuccessFlag',
           size = 'FreePlayers'
)

plt.xlabel("DistanceGoal")
plt.ylabel("MinDistance")
 
plt.show()

##### Data Standardization

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

##### Train/test split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split( 
    x, y, test_size=0.25, random_state=2023)

##### Implementation of oversampling and undersampling methods

In [ ]:
oversample = SMOTE(random_state = 2023)
xtrain_smote, ytrain_smote = oversample.fit_resample(xtrain, ytrain)

In [ ]:
oversample = ADASYN(random_state = 2023)
xtrain_ada, ytrain_ada = oversample.fit_resample(xtrain, ytrain)

In [ ]:
cc = ClusterCentroids(random_state = 2023)
xtrain_res, ytrain_res = cc.fit_resample(xtrain, ytrain)

### Perform a best subset search for logistic regression in terms of AUC score (Always outputs entire set)

In [ ]:
def fit_logistic_regression(features, target):
    features = sm.add_constant(features)
    model = sm.Logit(target, features)
    result = model.fit(disp=0)
    return result

def get_best_model(features, target, candidate_predictors):
    best_model = None
    best_auc = 0.0

    for subset in itertools.combinations(candidate_predictors, len(candidate_predictors)):
        current_features = features[:, list(subset)]
        result = fit_logistic_regression(current_features, target)
        y_pred_prob = result.predict(sm.add_constant(current_features))
        current_auc = roc_auc_score(target, y_pred_prob)

        if current_auc > best_auc:
            best_auc = current_auc
            best_model = result
            best_subset = subset

    return best_model, best_subset

# Create a list of predictor indices
num_predictors = xtrain.shape[1]
predictor_indices = list(range(num_predictors))

# Get the best model and subset based on AUC
best_model, best_subset = get_best_model(xtrain, ytrain, predictor_indices)

# Print the best subset and model summary
print("Best Subset:", best_subset)
print(best_model.summary(xname = ['Intercept']+list(data.drop(['SuccessFlag','Frame'],axis = 1).columns))) #To have the variable names, we use the fact that all variables are used

# Make predictions on the test set using the best subset
xtest_subset = xtest[:, list(best_subset)]
ypred_prob = best_model.predict(sm.add_constant(xtest_subset))

# Evaluate the AUC of the model on the test set
test_auc = roc_auc_score(ytest, ypred_prob)
print("Test AUC:", test_auc)

### Logistic regression fits

In [ ]:
classifier = LogisticRegression(random_state = 2023) 
classifier.fit(xtrain, ytrain)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 2023) 
classifier.fit(xtrain_smote, ytrain_smote)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 2023) 
classifier.fit(xtrain_ada, ytrain_ada)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 2023) 
classifier.fit(xtrain_res, ytrain_res)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

### Perform bayesian optimization over the xgboost hyperparameters with AUC as the scoring parameter

In [ ]:
estimators = [
    ('clf', XGBClassifier(random_state=123, device = 'cuda',tree_method = 'hist')) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)

In [ ]:
search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0),
    'clf__n_estimators': Integer(100,1000),
    'clf__n_jobs': Integer(11.5,12.5), #Admiteddly lazy way to get 12 processors working, adjust if needed
}

opt = BayesSearchCV(pipe, search_space, cv=10, n_iter=100, scoring='roc_auc', random_state=123)

In [ ]:
opt.fit(xtrain, ytrain)

In [ ]:
opt.best_estimator_

In [ ]:
opt.best_score_

In [ ]:
opt.score(xtest, ytest)

In [ ]:
opt.best_estimator_.steps

## Models obtained:

For RBF:

model = XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=0.5, colsample_bynode=1.0, colsample_bytree=0.5,
                device='cuda', early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=0.3404126229044155, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.003911225659823791,
                max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=8, max_leaves=None,
                min_child_weight=None, monotone_constraints=None,
                multi_strategy=None, n_estimators=1000, n_jobs=12,
                num_parallel_tree=None, random_state=123)

For KNN:

model = XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=0.8445304034569212, colsample_bynode=1.0,
                colsample_bytree=0.5, device='cuda', early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=4.733212503552665, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.48945402372274754,
                max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=2, max_leaves=None,
                min_child_weight=None, monotone_constraints=None,
                multi_strategy=None, n_estimators=1000, n_jobs=12,
                num_parallel_tree=None, random_state=123)

For hierarchical:

model = XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=0.5, colsample_bynode=1.0, colsample_bytree=1.0,
                device='cuda', early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=3.9022057058351827, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.001, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=8, max_leaves=None,
                min_child_weight=None, monotone_constraints=None,
                multi_strategy=None, n_estimators=1000, n_jobs=12,
                num_parallel_tree=None, random_state=123)

### XGBOOST fit

In [ ]:
model = XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=0.5, colsample_bynode=1.0, colsample_bytree=0.5, device='cuda', early_stopping_rounds=None, enable_categorical=False, eval_metric=None, feature_types=None, gamma=0.3404126229044155, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=0.003911225659823791, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=8, max_leaves=None, min_child_weight=None, monotone_constraints=None, multi_strategy=None, n_estimators=1000, n_jobs=12, num_parallel_tree=None, random_state=123)
model.fit(xtrain, ytrain)
y_pred = model.predict(xtest)
y_score = model.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))

## Analysis on feature influence

#### Feature importance (gini index)

In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(data.drop(['SuccessFlag','Frame'], axis=1).columns)[sorted_idx])
plt.title('Feature Importance')

#### SHAP values

In [ ]:
import shap

# explain the model's predictions using SHAP
explainer = shap.Explainer(model,feature_names = list(data.drop(['SuccessFlag','Frame'], axis=1).columns))
shap_values = explainer(xtest)

In [ ]:
shap.summary_plot(shap_values, xtest)

#### PDP plots

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

classifier = LogisticRegression(random_state = 2023) 
classifier.fit(xtrain, ytrain)
# Computing partial dependence plots
#features = list(range(14))
features = [11]
PartialDependenceDisplay.from_estimator(model, xtest, features, feature_names=list(data.drop(['SuccessFlag','Frame'], axis=1).columns),
                        n_jobs=3, grid_resolution=80)
#disp1 = PartialDependenceDisplay.from_estimator(model, xtest, features, feature_names=list(data.drop(['SuccessFlag','Frame'], axis=1).columns),
#                        n_jobs=3, grid_resolution=80, line_kw={"label": "XGBOOST"})
#disp2 = PartialDependenceDisplay.from_estimator(classifier, xtest, features, feature_names=list(data.drop(['SuccessFlag','Frame'], axis=1).columns),
#                        n_jobs=3, grid_resolution=80, ax = disp1.axes_, line_kw={"label": "Logistic Regression","color": "red"})
fig = plt.gcf()
fig.set_figwidth(10)
fig.set_figheight(10)
fig.subplots_adjust(wspace=1, hspace=0.5)